# Vytváření AI agentů s trvalou pamětí pomocí Cognee

Tento notebook ukazuje, jak vytvořit inteligentní AI agenty s pokročilými schopnostmi paměti pomocí [**cognee**](https://www.cognee.ai/) - open source AI paměti, která kombinuje znalostní grafy, sémantické vyhledávání a správu relací pro vytvoření kontextově uvědomělých AI systémů.

## 🎯 Cíle učení

Na konci tohoto tutoriálu budete rozumět, jak:
- **Vytvářet znalostní grafy podporované embeddingy**: Převádět nestrukturovaný text na strukturované, dotazovatelné znalosti
- **Implementovat paměť relací**: Vytvářet vícekrokové konverzace s automatickým uchováním kontextu
- **Ukládat konverzace**: Volitelně uchovávat důležité interakce v dlouhodobé paměti pro budoucí použití
- **Dotazovat se pomocí přirozeného jazyka**: Přistupovat k historickému kontextu a využívat ho v nových konverzacích
- **Vizualizovat paměť**: Prozkoumávat vztahy ve znalostním grafu vašeho agenta


## 🏗️ Co vytvoříte

V tomto tutoriálu vytvoříme **Asistenta pro programování** s trvalou pamětí, který:

### 1. **Vytváření znalostní báze**
   - Zpracovává informace o profilu a odbornosti vývojáře
   - Analyzuje principy a osvědčené postupy programování v Pythonu
   - Ukládá historické konverzace mezi vývojáři a AI asistenty

### 2. **Konverzace s povědomím o relaci**
   - Udržuje kontext napříč více otázkami v rámci jedné relace
   - Automaticky ukládá každou dvojici otázka/odpověď pro efektivní vyhledávání
   - Poskytuje koherentní a kontextové odpovědi na základě historie konverzace

### 3. **Dlouhodobá paměť**
   - Ukládá důležité konverzace do dlouhodobé paměti
   - Získává relevantní vzpomínky ze znalostní báze a minulých relací pro informování nových interakcí
   - Buduje rostoucí znalostní bázi, která se časem zlepšuje

### 4. **Inteligentní vyhledávání v paměti**
   - Používá grafově orientované sémantické vyhledávání k nalezení relevantních informací ve všech uložených znalostech
   - Filtruje vyhledávání podle podskupin dat (informace o vývojáři vs. principy)
   - Kombinuje více zdrojů dat pro poskytování komplexních odpovědí


## 📋 Předpoklady a nastavení

### Systémové požadavky

Před začátkem se ujistěte, že máte:

1. **Python prostředí**
   - Python 3.9 nebo novější
   - Virtuální prostředí (doporučeno)
   
2. **Redis Cache** (Požadováno pro správu relací)
   - Lokální Redis: `docker run -d -p 6379:6379 redis`
   - Nebo použijte spravovanou službu Redis
   
3. **Přístup k LLM API**
   - OpenAI API klíč nebo jiní poskytovatelé (viz [dokumentace](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Konfigurace databáze**
   - Ve výchozím nastavení není nutná žádná konfigurace. Cognee používá databáze založené na souborech (LanceDB a Kuzu)
   - Volitelně můžete nastavit Azure AI Search jako úložiště vektorů (viz [dokumentace](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Konfigurace prostředí

Vytvořte soubor `.env` ve vašem projektovém adresáři s následujícími proměnnými:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Pochopení paměťové architektury Cognee

### Jak Cognee funguje

Cognee poskytuje sofistikovaný paměťový systém, který přesahuje jednoduché ukládání klíč-hodnota:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Klíčové komponenty:

1. **Knowledge Graph**: Ukládá entity, vztahy a sémantické propojení
2. **Vector Embeddings**: Umožňuje sémantické vyhledávání ve všech uložených informacích
3. **Session Cache**: Udržuje kontext konverzace v rámci i mezi jednotlivými sezeními
4. **NodeSets**: Organizují data do logických kategorií pro cílené vyhledávání

### Typy paměti v tomto tutoriálu:

- **Persistent Memory**: Dlouhodobé ukládání v knowledge graphu
- **Session Memory**: Dočasný kontext konverzace v Redis cache
- **Semantic Memory**: Sémantické vyhledávání na základě podobnosti napříč všemi daty


## 📦 Instalace požadovaných balíčků

Nainstalujte Cognee s podporou Redis pro správu relací:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Inicializace prostředí a načtení knihoven

Ujistěte se, že:
1. Redis běží (např. pomocí Dockeru: `docker run -d -p 6379:6379 redis`)
2. Proměnné prostředí jsou nastaveny před importem modulů pro cache
3. Pokud je to potřeba, restartujte jádro a spusťte buňky v pořadí

Následující buňka provede:
1. Načtení proměnných prostředí z `.env`
2. Konfiguraci Cognee s vaším nastavením LLM
3. Povolení cache pro správu relací
4. Ověření, že všechny komponenty jsou správně propojeny


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Nastavení úložných adresářů

Cognee používá dva samostatné adresáře pro své operace:
- **Kořen dat**: Ukládá nahrané dokumenty a zpracovaná data
- **Kořen systému**: Obsahuje databázi znalostního grafu a systémová metadata

Pro tento návod vytvoříme oddělené adresáře následujícím způsobem:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Obnovení stavu paměti

Než začneme budovat náš paměťový systém, ujistěme se, že začínáme s čistým štítem.

> 💡 **Tip**: Tento krok můžete přeskočit, pokud si chcete zachovat stávající vzpomínky z předchozích spuštění při pozdějším použití tohoto notebooku.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Část 1: Vytváření znalostní báze

### Zdroje dat pro našeho asistenta vývojářů

Pro vytvoření komplexní znalostní báze budeme zpracovávat tři typy dat:

1. **Profil vývojáře**: Osobní odborné znalosti a technické zázemí
2. **Nejlepší postupy v Pythonu**: Zen Pythonu s praktickými pokyny
3. **Historické konverzace**: Minulé Q&A mezi vývojáři a AI asistenty

Tato různorodá data umožní našemu agentovi:
- Pochopit technický kontext uživatele
- Uplatňovat nejlepší postupy při doporučeních
- Učit se z předchozích úspěšných interakcí


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Zpracování dat do znalostního grafu

Nyní převedeme náš surový text do strukturované paměti. Tento proces:

1. **Přidává data do NodeSets**: Organizuje informace do logických kategorií
   - `developer_data`: Profil vývojáře a konverzace
   - `principles_data`: Nejlepší postupy a pokyny pro Python

2. **Spouští Cognify Pipeline**: Extrahuje entity, vztahy a vytváří vektory
   - Identifikuje klíčové koncepty
   - Vytváří sémantické spojení mezi souvisejícími informacemi
   - Generuje vektorové reprezentace

Tento proces může chvíli trvat, protože LLM zpracovává text a vytváří strukturu grafu:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Vizualizace znalostního grafu

Pojďme prozkoumat strukturu našeho znalostního grafu. Vizualizace ukazuje:
- **Uzly**: Entity extrahované z textu (koncepty, technologie, lidé)
- **Hrany**: Vztahy a spojení mezi entitami
- **Shluky**: Související koncepty seskupené podle sémantické podobnosti

Otevřete vygenerovaný HTML soubor ve svém prohlížeči a interaktivně prozkoumejte graf:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Obohaťte paměť s Memify

Funkce `memify()` analyzuje znalostní graf a generuje inteligentní pravidla o datech. Tento proces:
- Identifikuje vzory a osvědčené postupy
- Vytváří praktické pokyny na základě obsahu
- Navazuje vztahy mezi různými oblastmi znalostí

Tato pravidla pomáhají agentovi činit informovanější rozhodnutí při odpovídání na otázky. Zachycení druhé vizualizace vám umožní porovnat, jak se graf zahušťuje po obohacení.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Část 2: Inteligentní vyhledávání v paměti

### Ukázka 1: Integrace znalostí napříč dokumenty

Nyní, když je náš znalostní graf vytvořen, otestujeme, jak Cognee kombinuje informace z více zdrojů k zodpovězení složitých otázek.

První dotaz ukazuje:
- **Sémantické porozumění**: Nalezení relevantních konceptů, i když nejsou výslovně zmíněny
- **Křížové odkazování**: Kombinace profilu vývojáře se zásadami Pythonu
- **Kontekstuální uvažování**: Aplikace osvědčených postupů na konkrétní implementace

### Ukázka 2: Filtrované vyhledávání s NodeSets

Druhý dotaz ukazuje, jak cílit na konkrétní podmnožiny znalostního grafu:
- Používá parametr `node_name` k vyhledávání pouze v rámci `principles_data`
- Poskytuje zaměřené odpovědi z konkrétní znalostní oblasti
- Užitečné, když potřebujete informace specifické pro danou oblast


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Část 3: Nastavení správy relací

### Povolení paměti konverzace

Správa relací je klíčová pro udržení kontextu napříč více interakcemi. Zde provedeme:

1. **Inicializace uživatelského kontextu**: Vytvoření nebo načtení uživatelského profilu pro sledování relací
2. **Konfigurace cache enginu**: Připojení k Redis pro ukládání historie konverzací
3. **Povolení proměnných relace**: Nastavení kontextových proměnných, které přetrvávají mezi dotazy

> ⚠️ **Důležité**: To vyžaduje spuštěný Redis a `CACHING=true` ve vašem prostředí


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Pomocná funkce: Zobrazení historie relace

Tato užitečná funkce nám umožňuje prohlížet historii konverzace uloženou v Redis. Je užitečná pro:
- Ladění správy relací
- Ověření, že konverzace jsou ukládány do mezipaměti
- Pochopení, jaký kontext je agentovi k dispozici


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Sezení 1: Laboratoř podpory asynchronního programování — První otázka

Začněte sezení `async-support-lab` položením otázky na telemetrii přívětivé vzory asyncio pro masivní webový scraper. Graf už zná asyncio, aiohttp a monitorovací postupy, takže odpověď by měla odrážet předchozí konverzace a zároveň přizpůsobit odpověď novému dotazu.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Kontrola paměti relace 1 po první výměně

Spuštění `show_history(session_1)` ihned po úvodní otázce potvrdí, že Cognee zapsal jak zadání, tak odpověď do Redis. Měli byste vidět jeden záznam s pokyny pro souběžnost.


In [ ]:
await show_history(session_1)

## Sezení 1: Následná diskuse o datových modelech

Dále se ptáme: „Kdy bych měl zvolit dataclasses oproti Pydantic?“ s použitím stejného ID sezení. Cognee by měl propojit principy Pythonu spolu s předchozími diskusemi o FastAPI, aby poskytl detailní rady – ukazující, že kontext se přenáší v rámci pojmenovaného sezení.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Potvrďte, že historie relace 1 obsahuje oba kroky

Další volání `show_history(session_1)` by mělo odhalit dva záznamy otázek a odpovědí. To odpovídá kroku "přehrání paměti" v laboratoři Mem0 a dokazuje, že další kroky rozšiřují stejný přepis.


In [ ]:
await show_history(session_1)

## Relace 2: Vlákno pro kontrolu návrhu — Nová relace

Abychom ukázali izolaci mezi vlákny, spouštíme `design-review-session` a žádáme o pokyny k protokolování pro přezkumy incidentů. I když je základní znalostní báze stejná, nové ID relace udržuje přepisy oddělené.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Revize relace 2 Historie

`show_history(session_2)` by měl zobrazit pouze dvojici výzva/odpověď z designové revize. Porovnejte to s relací 1, abyste zdůraznili, jak Cognee udržuje nezávislé přepisy a zároveň znovu využívá sdílený znalostní graf.


In [ ]:
await show_history(session_2)

## Shrnutí

Gratulujeme! Právě jste svému programátorskému asistentovi přidali skutečnou vrstvu dlouhodobé paměti poháněnou Cognee.

V tomto tutoriálu jste vzali surový obsah pro vývojáře (kód, dokumentaci, chaty) a přeměnili jej na graf + vektorovou paměť, kterou váš agent může prohledávat, analyzovat a neustále zlepšovat.

Co jste se naučili

1. **Od surového textu k AI paměti**: Jak Cognee zpracovává nestrukturovaná data a přeměňuje je na inteligentní, prohledávatelnou paměť pomocí kombinované architektury vektorů a znalostního grafu.

2. **Obohacení grafu pomocí memify**: Jak jít nad rámec základního vytváření grafu a použít memify k přidání odvozených faktů a bohatších vztahů do vašeho stávajícího grafu.

3. **Různé strategie vyhledávání**: Jak dotazovat paměť pomocí různých typů vyhledávání (dotazy s povědomím o grafu, dokončování ve stylu RAG, přehledy, surové části, vyhledávání kódu atd.) podle potřeb vašeho agenta.

4. **Vizualizace a průzkum**: Jak zkontrolovat a ladit, co Cognee vytvořilo, pomocí vizualizací grafů a uživatelského rozhraní Cognee, abyste mohli skutečně vidět, jak je znalost strukturována.

5. **Paměť s ohledem na relace**: Jak kombinovat kontext jednotlivých relací s trvalou sémantickou pamětí, aby si agenti mohli pamatovat mezi jednotlivými spuštěními, aniž by docházelo k úniku informací mezi uživateli.


## Klíčové poznatky
1. Paměť jako znalostní graf podporovaný vektory

    - **Strukturované porozumění**: Cognee kombinuje úložiště vektorů a grafové úložiště, takže vaše data jsou vyhledatelná jak podle významu, tak podle vztahů. Cognee standardně používá databáze založené na souborech (LanceDB pro vektorové databáze, Kuzu pro grafové databáze).

    - **Vyhledávání s ohledem na vztahy**: Odpovědi mohou být založeny nejen na „podobném textu“, ale také na tom, jak spolu entity souvisejí.

    - **Živá paměť**: Vrstva paměti se vyvíjí, roste a zůstává dotazovatelná jako jeden propojený graf.

2. Režimy vyhledávání a uvažování
    - **Hybridní vyhledávání**: Vyhledávání kombinuje podobnost vektorů, strukturu grafu a uvažování LLM, od vyhledávání surových částí až po dotazování s ohledem na graf.

    - **Přizpůsobení režimu úkolu**: Používejte režimy ve stylu dokončování, když chcete odpovědi v přirozeném jazyce, a režimy částí/souhrnů/grafů, když váš agent potřebuje surový kontext nebo chce řídit své vlastní uvažování.

3. Personalizovaní agenti s povědomím o relaci
    - **Kontext relace + dlouhodobá paměť**: Cognee odděluje krátkodobý kontext „vlákna“ od dlouhodobé paměti na úrovni uživatele nebo organizace.

## Reálné aplikace

1. **Vertikální AI agenti**

    Použijte vzor z tohoto notebooku k vytvoření doménově inteligentních kopilotů, kteří využívají Cognee jako jádro pro vyhledávání a uvažování:

- **Kopiloti pro vývojáře**: Asistenti pro kontrolu kódu, analýzu incidentů a architekturu, kteří procházejí kód, API, návrhové dokumenty a tikety jako jeden paměťový graf.

- **Kopiloti pro zákazníky**: Podpora nebo asistenti úspěchu, kteří čerpají z produktové dokumentace, FAQ, poznámek CRM a minulých tiketů s vyhledáváním s ohledem na graf a citovanými odpověďmi.

- **Interní expertní kopiloti**: Asistenti pro politiku, právo nebo bezpečnost, kteří uvažují nad propojenými pravidly, směrnicemi a historickými rozhodnutími místo izolovaných PDF.

    Cognee je výslovně navrženo jako trvalá, přesná paměť pro AI agenty, poskytující živý znalostní graf, který se integruje za vašeho agenta a nahrazuje ad-hoc kombinace úložišť vektorů a vlastního grafového kódu.

2. **Sjednocení datových sil do jedné paměti**

    Stejný přístup vám také pomůže vytvořit sjednocenou paměťovou vrstvu napříč rozptýlenými zdroji:

- **Od sil k jednomu grafu**: Načtěte strukturovaná (např. databáze) a nestrukturovaná data (např. dokumenty, chaty) do jednoho grafu podporovaného vektory, místo oddělených indexů pro každý systém.

- **Mezi-zdrojové uvažování s citacemi**: Proveďte vícekrokové uvažování nad vším – „propojte“ logy, metriky a dokumenty prostřednictvím grafu – a stále vracejte odpovědi s doložením původu.

- **Centra znalostí**: Pro oblasti jako bankovnictví nebo vzdělávání je Cognee již využíváno k sjednocení PDF, interních systémů a dat aplikací do jednoho znalostního grafu s vektory, aby agenti mohli odpovídat na otázky s přesným, citovaným kontextem.

## Další kroky

Implementovali jste základní paměťovou smyčku. Zde jsou přirozená rozšíření, která si můžete vyzkoušet sami (viz [dokumentace Cognee](https://docs.cognee.ai/) pro podrobnosti):

1. **Experimentujte s časovým povědomím**: Zapněte časové zpracování, abyste extrahovali události a časové značky z textu.

2. **Zaveďte uvažování řízené ontologií**: Definujte OWL ontologii pro vaši doménu. Použijte podporu ontologií Cognee, aby extrahované entity a vztahy byly ukotveny v tomto schématu, což zlepší kvalitu grafu a odpovědi specifické pro danou doménu.

3. **Přidejte zpětnovazební smyčku**: Nechte Cognee upravovat váhy hran grafu na základě skutečné zpětné vazby uživatelů, aby se vyhledávání časem zlepšovalo místo toho, aby zůstalo statické.

4. **Vylaďte personalizaci a chování relací**: Použijte ID uživatelů, nájemce a datové sady, abyste každé osobě nebo týmu poskytli vlastní pohled na sdílený paměťový engine.

5. **Rozšiřte na složitější agenty**: Připojte Cognee k rámcům agentů, abyste vytvořili systémy s více agenty, které všechny sdílejí stejnou paměťovou vrstvu. *Microsoft Agent Framework x Cognee plugin brzy k dispozici.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Prohlášení**:  
Tento dokument byl přeložen pomocí služby AI pro překlady [Co-op Translator](https://github.com/Azure/co-op-translator). I když se snažíme o přesnost, mějte prosím na paměti, že automatizované překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace se doporučuje profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
